In [ ]:
!pip install lightning torchvision pytorchvideo einops

In [1]:
try:
    from key_utils import KeySegmentDataModule, KeyClf, id2label, label2id
except:
    import sys
    sys.path.append("/kaggle/input/keystroke-util")
    from key_utils import KeySegmentDataModule, Key Clf, id2label, label2id

from lightning.pytorch.callbacks import EarlyStopping
from pytorchvideo.models.x3d import create_x3d
import torchvision
import torchvision.transforms.functional
import lightning as L
import torch


def transforms(frames):
    h, w = frames.shape[-2], frames.shape[-1]

    frames = torchvision.transforms.functional.resized_crop(
                              frames, 
                              top=h//2, 
                              left=0, 
                              height=h//2,
                              width=w,
                              size=(224, 224))
    
    frames = frames.permute(1, 0, 2, 3)
    frames = frames / 255.0
    return frames.float()
    
dm = KeySegmentDataModule(segment_dir='datasets/angle/segments_dir', 
                          num_workers=0,
                          transforms=transforms,
                          batch_size=2)
weights = dm.train_weights


class X3DKeyClf(KeyClf):
    def __init__(self, learning_rate=0.01):
        super().__init__(weights, learning_rate)
        self.model = create_x3d(
            input_channel=3, 
            input_clip_length=8, 
            input_crop_size=224,
            model_num_class=len(label2id)
        )



module = X3DKeyClf()
trainer = L.Trainer(
    # deterministic=True,
    # devices=[0, 1],
    accelerator="gpu",
    fast_dev_run=False,
    max_epochs=100,
    callbacks=EarlyStopping(monitor='val_loss', patience=5),
)

trainer.fit(module, dm)
# trainer.test(module, dm)

/Users/haily/.pyenv/versions/3.10.4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Train:
 Counter({'idle': 2715, 'space': 917, 'e': 497, 'BackSpace': 429, 'i': 328, 'a': 320, 'o': 302, 't': 289, 'r': 250, 'n': 246, 's': 215, 'u': 184, 'l': 183, 'h': 162, 'd': 159, 'c': 155, 'y': 119, 'g': 109, 'm': 109, 'p': 108, 'w': 103, 'b': 91, 'k': 86, 'f': 85, 'dot': 84, 'v': 73, 'comma': 66, 'j': 62, 'z': 58, 'x': 54, 'q': 52})
Val:
 Counter({'idle': 973, 'space': 311, 'e': 162, 'BackSpace': 136, 'i': 112, 'a': 108, 't': 96, 'o': 87, 'n': 75, 'r': 73, 'h': 67, 's': 62, 'u': 57, 'l': 52, 'c': 49, 'd': 49, 'f': 43, 'y': 41, 'm': 39, 'g': 38, 'w': 28, 'p': 26, 'comma': 26, 'b': 26, 'z': 24, 'dot': 23, 'v': 22, 'x': 18, 'k': 16, 'j': 16, 'q': 15})
Test:
 Counter({'idle': 944, 'space': 316, 'BackSpace': 164, 'e': 147, 'i': 112, 't': 103, 'o': 94, 'n': 85, 'r': 84, 'a': 80, 's': 64, 'l': 62, 'c': 57, 'u': 54, 'm': 53, 'd': 46, 'h': 42, 'w': 37, 'f': 36, 'y': 36, 'g': 33, 'p': 31, 'b': 31, 'dot': 28, 'k': 23, 'v': 23, 'q': 21, 'comma': 20, 'x': 20, 'z': 13, 'j': 11})
train_weights: 

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/haily/Documents/GitHub/Research Learning/lightning_logs

  | Name      | Type               | Params | Mode 
---------------------------------------------------------
0 | loss_fn   | CrossEntropyLoss   | 0      | train
1 | train_acc | MulticlassAccuracy | 0      | train
2 | val_acc   | MulticlassAccuracy | 0      | train
3 | test_acc  | MulticlassAccuracy | 0      | train
4 | model     | Net                | 3.0 M  | train
---------------------------------------------------------
3.0 M     Trainable params
0         Non-trainable params
3.0 M     Total params
12.153    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/haily/.pyenv/versions/3.10.4/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]